<a href="https://colab.research.google.com/github/yussif-issah/css54FinalProject/blob/main/mangrove_parameter_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -Uq

In [ ]:
import wandb

ModuleNotFoundError: No module named 'wandb'

In [ ]:
wandb.login()

True

In [ ]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'val_accuracy',
    'goal': 'maximize'
    }

sweep_config['metric'] = metric
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer1_size': {
        'values': [128, 256, 512]
        },

    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict
parameters_dict.update({
    'epochs': {
        'value': 10}
    })
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.0001,
        'max': 0.01
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': ,
      }
    })


In [ ]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 64,
                               'min': 12,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 10},
                'fc_layer1_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.01,
                                  'min': 0.0001},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="assignment3-sweeps")

Create sweep with ID: 3in1u5mq
Sweep URL: https://wandb.ai/gyanko/assignment3-sweeps/sweeps/3in1u5mq


In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
NUM_CLASSES = 101
TRAINING_DATA_PATH = "/content/drive/MyDrive/train_files"
LABELS_FILE_PATH = "/content/drive/MyDrive/labels.txt"

In [ ]:
def buildModel(fc_layer1,drop_out):
  model = models.mobilenet_v3_small(pretrained=True)
  head = nn.Sequential(
        nn.Linear(model.classifier[-1].in_features, 256),
        nn.ReLU(),
        nn.Dropout(drop_out),
        nn.Linear(fc_layer1, NUM_CLASSES)
  )

  model.classifier[-1] = head
  return model.to(device)

In [ ]:
def buildOptimizer(model,optimizer,learning_rate):
  if optimizer == "sgd":
        optimizer = optim.SGD(model.parameters(),
                              lr=learning_rate, momentum=0.9)
  elif optimizer == "adam":
        optimizer = optim.Adam(model.parameters(),
                               lr=learning_rate)
  return optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_folder, label_file, transform=None):
        self.img_folder = img_folder
        self.label_file = label_file
        self.transform = transform
        self.data = self._read_data()

    def _read_data(self):
        data = []
        with open(self.label_file, 'r') as f:
            labels = [int(label.strip()) for label in f.readlines()]
        img_files = sorted(os.listdir(self.img_folder))
        for img_file, label in zip(img_files, labels):
            img_path = os.path.join(self.img_folder, img_file)
            data.append((img_path, label))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
def buildDataLoader(batchSize,image_folder, label_file):
  ttransform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
  whole_data = CustomDataset(image_folder,label_file,ttransform)

  generator1 = torch.Generator().manual_seed(42)
  generator2 = torch.Generator().manual_seed(32)

  tune_size = int(0.1*len(whole_data))
  rest_size = len(whole_data) - tune_size

  training_size = int(0.8*tune_size)
  val_size = tune_size - training_size

  parameter_tuning_dataset,_ = random_split(whole_data,[tune_size, rest_size], generator=generator1)
  train_data, val_data = random_split(parameter_tuning_dataset,[training_size, val_size], generator=generator2)


  train_data_loader = DataLoader(train_data,batch_size=batchSize,shuffle=True)
  val_data_loader = DataLoader(val_data,batch_size=64,shuffle=True)

  image_datasets = {"train":train_data,"val":val_data}
  data_loaders = {"train": train_data_loader,"val":val_data_loader}

  return data_loaders,image_datasets

In [ ]:
def trainModel(config=None):

  with wandb.init(config=config):

    config = wandb.config
    data_loaders,datasets = buildDataLoader(config.batch_size,TRAINING_DATA_PATH,LABELS_FILE_PATH)
    model = buildModel(config.fc_layer1_size,config.dropout)
    optimizer = buildOptimizer(model,config.optimizer,config.learning_rate)

    for epoch in range(config.epochs):

      for phase in ["train","val"]:

        if phase == "train":
          model.train()
        else:
          model.eval()

        running_loss = 0
        correct = 0

        for inputs,labels in data_loaders[phase]:
          inputs,labels = inputs.to(device),labels.to(device)

          optimizer.zero_grad()

          with torch.set_grad_enabled(phase=="train"):
            outputs = model(inputs)
            _,preds = torch.max(outputs,1)
            loss = criterion(outputs,labels)

            if phase == "train":
              loss.backward()
              optimizer.step()

          running_loss += loss.item()*inputs.size(0)
          correct += torch.sum(preds ==labels)

          epoch_loss = running_loss / len(datasets[phase])
          epoch_acc = correct.double() / len(datasets[phase])

          if phase == "train":
            wandb.log({"epoch": epoch,"training loss": epoch_loss, "training accuracy":epoch_acc})
          else:
            wandb.log({"epoch": epoch,"validation loss": epoch_loss, "validation accuracy":epoch_acc})

In [ ]:
wandb.agent(sweep_id, trainModel, count=5)

wandb: Agent Starting Run: cfct7s29 with config:
wandb: 	batch_size: 56
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	fc_layer1_size: 128
wandb: 	learning_rate: 0.0024356831129314143
wandb: 	optimizer: adam


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
training accuracy,▁▁▂▃▁▂▃▄▁▂▃▅▁▃▄▅▂▃▅▆▁▃▅▇▂▃▅▇▂▄▅▇▂▄▆▇▂▄▆█
training loss,▂▄▆█▁▃▄▆▁▃▄▅▁▂▄▄▁▂▃▄▁▂▃▄▁▂▃▃▁▂▂▃▁▂▂▃▁▁▂▃
validation accuracy,▁▅▆▁▅▅▁▅▆▁▆▇▂▆▇▁▅▆▁▆▇▂▆▇▃▇█▁▅▆
validation loss,▁▄▅▁▃▄▂▅▇▃▇█▂▄▅▂▆▇▂▅▇▂▅▆▂▅▇▂▅▆
epoch,9
training accuracy,0.77265
training loss,0.83765
validation accuracy,0.30612
validation loss,5.96748


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 65gos7qe with config:
wandb: 	batch_size: 48
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	fc_layer1_size: 64
wandb: 	learning_rate: 0.005914055837625587
wandb: 	optimizer: sgd


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


...

wandb: Agent Starting Run: gz5b9z8h with config:
wandb: 	batch_size: 88
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	fc_layer1_size: 512
wandb: 	learning_rate: 0.008457124463952864
wandb: 	optimizer: adam


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
training accuracy,▁▂▂▄▂▃▄▆▂▂▄▅▂▄▄▆▂▄▅▇▂▄▅▇▂▄▆▇▂▄▆▇▂▅▇█▂▄▆█
training loss,▂▃▆█▂▃▅▇▁▂▅▇▁▃▅▇▁▃▄▆▁▃▄▆▁▃▅▆▁▃▅▆▁▃▅▅▁▃▅▆
validation accuracy,▅▇█▁▅▇▁▅█▂▄▅▂▅▅▂▇█▂▇█▂▅█▂▆▇▂▇▇
validation loss,▁▂▃▁▂▂▃▅▅▂▄▄▃▄▅▄▇█▄▇▇▁▂▂▂▃▃▁▂▂
epoch,9
training accuracy,0.30769
training loss,2.96958
validation accuracy,0.09524
validation loss,473.94482


wandb: Agent Starting Run: nuncm0uw with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.4
wandb: 	epochs: 10
wandb: 	fc_layer1_size: 512
wandb: 	learning_rate: 0.000339223951539606
wandb: 	optimizer: adam


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
training accuracy,▁▁▂▂▁▂▂▃▁▂▃▄▁▃▄▅▂▃▅▆▂▃▅▆▂▄▅▇▂▃▆▇▃▄▅█▂▄▆█
training loss,▂▄▆█▁▃▅▆▁▃▄▅▁▂▃▄▁▂▃▃▁▂▂▃▁▂▂▂▁▁▂▂▁▁▂▂▁▁▂▂
validation accuracy,▁▃▃▁▃▃▁▄▄▂▆▇▂▅▆▃▆▇▃▆▇▃▇█▂▆▇▃▆█
validation loss,▃▇█▃▆▇▂▆▇▂▅▅▂▄▅▁▃▄▁▄▄▁▃▄▁▃▄▁▃▃
epoch,9
training accuracy,0.87863
training loss,0.52381
validation accuracy,0.60544
validation loss,1.66679


wandb: Agent Starting Run: v2z5ppbz with config:
wandb: 	batch_size: 40
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	fc_layer1_size: 64
wandb: 	learning_rate: 0.004218410109900594
wandb: 	optimizer: adam
